# SMAI Assignment - 2

## Question - `3` : Multinomial Naïve Bayes

| | |
|- | -|
| Course | Statistical Methods in AI |
| Release Date | `16.02.2023` |
| Due Date | `24.02.2023` |

This question will have you working and experimenting with the Multinomial Naïve Bayes classifier. Initially, you will transform the given data in csv file to count matrix, then calculate the priors. Use those priors to compute likelihoods according to Multinomial Naive Bayes and then classify the test data. Please note that use of `sklearn` implementations is only for the final question of the assignment, for other doubts regarding libraries you can reach out to the TAs.

The dataset is about `Spam SMS`. There is 1 attribute that is the `message`, and the class label which could be `spam` or `ham`. The data is present in `spam.csv`. It contains about 5-6000 samples.
For your convinience the data is already pre-processed and loaded, but I suggest you to just take a look at the code for your own knowledge, and parts vectorization is left up to you which could be easily done with the help of the given example code.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

## Reading text-based data using pandas

In [3]:
# read file into pandas using a relative path

df = pd.read_csv("./spam.csv", encoding='latin-1')
df.dropna(how="any", inplace=True, axis=1)
df.columns = ['label', 'message']

df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Pre-processing

- Our main issue with our data is that it is all in text format (strings). The classification algorithms that we usally use need some sort of numerical feature vector in order to perform the classification task. There are actually many methods to convert a corpus to a vector format. The simplest is the bag-of-words approach, where each unique word in a text will be represented by one number.

- As a first step, let's write a function that will split a message into its individual words and return a list. We'll also remove very common words, ('the', 'a', etc..). To do this we will take advantage of the NLTK library. It's pretty much the standard library in Python for processing text and has a lot of useful features. We'll only use some of the basic ones here.

In [4]:
import string
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    STOPWORDS = stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure']
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return ' '.join([word for word in nopunc.split() if word.lower() not in STOPWORDS])

[nltk_data] Downloading package stopwords to /home/gagan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df['message'] = df.message.apply(text_process)
df.head()

,label,message
0,ham,Go jurong point crazy Available bugis n great ...
1,ham,Ok lar Joking wif oni
2,spam,Free entry wkly comp win FA Cup final tkts 21s...
3,ham,dun say early hor c already say
4,ham,Nah think goes usf lives around though


In [6]:
df['label'] = df.label.map({'ham':0, 'spam':1})
df.head()

,label,message
0,0,Go jurong point crazy Available bugis n great ...
1,0,Ok lar Joking wif oni
2,1,Free entry wkly comp win FA Cup final tkts 21s...
3,0,dun say early hor c already say
4,0,Nah think goes usf lives around though


## Splitting the data

In [7]:
def vectoriseX(X):
    cv = CountVectorizer()
    X = cv.fit_transform(X).toarray()
    return X

In [8]:
# split X and y into training and testing sets 
from sklearn.model_selection import train_test_split

X = df.message
y = df.label

print(f'X: {X.shape}')
print(f'y: {y.shape}')
print()

# X = vectoriseX(X)
y = y.ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

print(f'X_train: {X_train.shape}')
print(f'y_train: {y_train.shape}')
print()

print(f'X_test: {X_test.shape}')
print(f'y_test: {y_test.shape}')
print()

X: (5572,)
y: (5572,)

X_train: (4179,)
y_train: (4179,)

X_test: (1393,)
y_test: (1393,)



## Helper code / Example code for Representing text as Numerical data using Sci-kit learn

📌 From the scikit-learn documentation:
- Text Analysis is a major application field for machine learning algorithms. However the raw data, a sequence of symbols cannot be fed directly to the algorithms themselves as most of them expect numerical feature vectors with a fixed size rather than the raw text documents with variable length.
- We will use CountVectorizer to "convert text into a matrix of token counts":

In [9]:
# example text for model training (SMS messages)
simple_train = ['call you tonight', 'Call me a cab', 'Please call me... PLEASE!']

In [10]:
# import and instantiate CountVectorizer (with the default parameters)
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()
simple_train = vect.fit_transform(simple_train)

vect.get_feature_names_out()

array(['cab', 'call', 'me', 'please', 'tonight', 'you'], dtype=object)

In [11]:
vect.get_feature_names_out()

array(['cab', 'call', 'me', 'please', 'tonight', 'you'], dtype=object)

In [12]:
# convert sparse matrix to a dense matrix
simple_train.toarray()

array([[0, 1, 0, 0, 1, 1],
       [1, 1, 1, 0, 0, 0],
       [0, 1, 1, 2, 0, 0]])

In this scheme, features and samples are defined as follows:

- Each individual token occurrence frequency (normalized or not) is treated as a feature.
- The vector of all the token frequencies for a given document is considered a multivariate sample.

A corpus of documents can thus be represented by a matrix with one row per document and one column per token (e.g. word) occurring in the corpus.

In [13]:
# examine the vocabulary and document-term matrix together
pd.DataFrame(simple_train.toarray(), columns=vect.get_feature_names_out())

,cab,call,me,please,tonight,you
0,0,1,0,0,1,1
1,1,1,1,0,0,0
2,0,1,1,2,0,0


In [14]:
X = vectoriseX(X)
y = y.ravel()
x_train, x_test, Y_train, Y_test = train_test_split(X, y, test_size=0.25, random_state=1)

### Transform Testing data into a document-term matrix (using existing / training vocabulary)

- You are supposed to use the training vocabolary to make the count matrix for test data

In [15]:
simple_test = ["please don't call me"]

In [16]:
simple_test_dtm = vect.transform(simple_test)
simple_test_dtm.toarray()

array([[0, 1, 1, 1, 0, 0]])

In [17]:
# examine the vocabulary and document-term matrix together
pd.DataFrame(simple_test_dtm.toarray(), columns=vect.get_feature_names_out())

,cab,call,me,please,tonight,you
0,0,1,1,1,0,0


## Multinomial Naive Bayes Implementation

- Your task is to implement Mutlinomial Naive Bayes from scratch, you can use numpy to vectorize your code and matplotlib  to show your analysis.
- Below some information has given from the documentation about Multinomial Naive Bayes, this will give you some idea about using *Smoothing Priors*.
- There is a sub-question for experimenting with $\alpha > 0$, you don't have to implement it separetely, try to incomporate it in same Model Class / Function.

📌 From the scikit-learn documentation:

- Multinomial Naive Bayes implements the naive Bayes algorithm for multinomially distributed data, and is one of the two classic naive Bayes variants used in text classification (where the data are typically represented as word vector counts, although tf-idf vectors are also known to work well in practice).

- The distribution $\theta_y = (\theta_{y1}, \theta_{y2}, \dots, \theta_{yn})$ is parametrized by vectors for each class $y$, where $n$ is the number of features (in text classification, the size of the vocabulary) and $\theta_{yi}$ is the probability $P(x_i|y)$ of feature appearing in a sample belonging to class.

- The parameters $\theta_y$ is estimated by a smoothed version of maximum likelihood, i.e. relative frequency counting:

$$
\hat{\theta}_{yi} = \frac{N_{yi} + \alpha}{N_{y} + \alpha n}
$$

 where $N_{yi} = \sum_{x \in T}{x_i}$ is the number of times feature $i $ appears in a sample of class in the training set $T$, and $N_{y} = \sum^{n}_{i=1}{N_{yi}}$ is the total count of all features for class $y$.

- The smoothing priors $\alpha \gt 0$ accounts for features not present in the learning samples and **prevents zero probabilities** in further computations. Setting $\alpha = 1$ is called Laplace smoothing, while $\alpha \lt 1$ is called Lidstone smoothing.


In [18]:
"""
Your code here
"""

'\nYour code here\n'

## Vectorizing Training Sample

- Use the Helper code above to vectorize for training samples
- Don't overthink it, its very easy to do

In [19]:
class MultiNB:
    def __init__(self, alpha=1):
        self.alpha = alpha
    
    def prior(self): 
        p = np.zeros(self.total_class_c)
        _, dist = np.unique(self.y, return_counts=True)
        for i in range(self.classes_.shape[0]):
            p[i] = dist[i] / self.sample_count
        return p
            
    def fit(self, X, y): 
        self.y = y
        self.sample_count, self.feature_count = X.shape
        self.classes_ = np.unique(y)
        self.total_class_c = self.classes_.shape[0]
        self.prior_probab = self.prior()
        
        self.uniques = [np.unique(X[:, i]) for i in range(self.feature_count)]
            
        self.n_yi = np.zeros((self.total_class_c, self.feature_count)) 
        self.n_y = np.zeros(self.total_class_c) 
        for i in self.classes_:
            # indices = np.argwhere(self.y == i).flatten()
            self.n_yi[i] = np.sum(X[np.argwhere(self.y == i).flatten()], axis=0) 
            self.n_y[i] = np.sum(self.n_yi[i]) 
            
    def theta(self, x_i, i, h):
        return ((self.n_yi[h, i] + self.alpha) / (self.n_y[h] + (self.alpha * self.feature_count))) ** x_i
    
    def likelihood(self, x, h):
        return np.prod([self.theta(x[i], i, h) for i in range(x.shape[0])])
    
    def predict(self, X):
        samples = X.shape[0]
        self.predict_proba = np.zeros((samples, self.total_class_c))
        for i in range(samples):
            overall_likelihood = [self.prior_probab[h] * self.likelihood(X[i], h) for h in range(self.total_class_c)]
            self.predict_proba[i] = overall_likelihood / np.sum(overall_likelihood)
            
        # indices = 
        return self.classes_[np.argmax(self.predict_proba, axis=1)]


In [20]:
"""
Your code here
"""


'\nYour code here\n'

## Calculate Priors and Estimate Model's performance on Training Sample

- Calculate priors based on Training Sample using your NB implementation
- Evaluate your model's performance on Training Data ($\alpha = 0$)

In [21]:
cv = CountVectorizer()
X_train = cv.fit_transform(X_train).toarray()

In [22]:
X_test = cv.transform(X_test).toarray()

In [23]:
print(X_test.shape)
print(X_train.shape)

(1393, 7996)
(4179, 7996)


## Vectorizing Test Sample

- Use the Training Sample vocabulary to create word count matrix for test samples
- This is also shown in the Helper code

## Estimate Model's performance on Test Sample

- Evaluate your model's performance on Test Sample, using the Training Priors ($\alpha = 0$)

## Select Smoothing Priors

- Refactor your code to incorporate smoothing priors, select $\alpha = 0$ for the previous estimates / sub-questions
- Compare the performance with different values of $\alpha \gt 0$ as smoothing priors to take care of zero probabilities
- You can display a Plot or Table to show the comparison.

In [27]:
"""
Your code here
"""
li = [1,2,3,4]
accuracy_score_list = []
for i in li:
    me = MultiNB(i)
    me.fit(x_train, Y_train)
    y_pred = me.predict(x_test)
    accuracy_score_list.append(accuracy_score(Y_test,y_pred))
    print(f'alpha: {i}, accuracy: {accuracy_score(Y_test,y_pred)}')

alpha: 1, accuracy: 0.9763101220387652
alpha: 2, accuracy: 0.9784637473079684
alpha: 3, accuracy: 0.9784637473079684
alpha: 4, accuracy: 0.9777458722182341


## Comparison with Sci-kit Learn Implementation

- Use sci-kit learn's `sklearn.naive_bayes.MultinomialNB` model to compare your implementation's performance
- (Optional) try other classifiers from `sklearn.naive_bayes` and see if you can make them work`

In [29]:
"""
Your code here
"""
from sklearn.naive_bayes import MultinomialNB
li = [1,2,3,4]
accuracy_score_list_sk = []

for i in li:
    clf = MultinomialNB(force_alpha=True,alpha=i)
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    accuracy_score_list_sk.append(clf.score(X_test,y_test))

In [45]:
print('Aplha\tMNB \t\t\tSklearnMNB')
print('-----\t------------------\t------------------')

# Loop over the lists and print each row
c = 1
for ac, acsk in zip(accuracy_score_list, accuracy_score_list_sk):
    print(f'{c}\t{ac}\t{acsk}')
    c = c + 1

Aplha	MNB 			SklearnMNB
-----	------------------	------------------
1	0.9763101220387652	0.9827709978463748
2	0.9784637473079684	0.9834888729361091
3	0.9784637473079684	0.9806173725771715
4	0.9777458722182341	0.9806173725771715
